In [ ]:
import numpy as np
import os
import math
import torch

In [4]:
DEFAULT_CONFIG = {
    'MAX_TEXT_LENGTH' : 128,

    'VOCAB' : "./datasets/pre-process/captions_vocab_bird.pickle",      # The path to DAMSM vocab pickle file
    'LANGUAGE_ENCODER' : "./datasets/text_encoder_bird.pth",    # The path to DAMSM text encoder
    'IMAGE_ENCODER': "./datasets/image_encoder_bird.pth",   # The path to DAMSM image encoder
    'EMBEDDING_DIM' : 256,

    'CATE_IMAGE_TRAIN' : "./datasets/CUB_200_2011/cate_image_train.json",   # The path to category-image mapping cache file
    'IMAGE_CATE_TRAIN' : "./datasets/CUB_200_2011/image_cate.json",     # The path to image-category mapping cache file

    'CAPTION' : "./datasets/CUB_200_2011/caption.json",     # The path to image-caption cache file

}

In [5]:
import subprocess
import pickle
# Replace 'your_pipe_file' with the path to your pipe file
pipe_file_path = DEFAULT_CONFIG['VOCAB']

# try:
#     # Open the pipe file for reading
#     with open(pipe_file_path, 'rb', buffering=0) as pipe:
#         # Read the content from the pipe
#         content = pipe.read(10024)
#         print("Content of the pipe file:")
#         print(content)
# except FileNotFoundError:
#     print("Pipe file not found.")
# except Exception as e:
#     print("An error occurred:", e)

objects = []
with (open(pipe_file_path, "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [13]:
len(objects[0][0])

88550

In [31]:
objects[0][2]

{0: '<end>',
 1: 'fawn',
 2: 'blsck',
 3: 'smallperching',
 4: 'seated',
 5: 'headand',
 6: 'mandible',
 7: 'perwinkle',
 8: 'yellos',
 9: 'comparatively',
 10: 'dynamic',
 11: 'outlining',
 12: 'yellow',
 13: 'sleek',
 14: 'four',
 15: 'hip',
 16: 'sleep',
 17: 'hanging',
 18: 'consists',
 19: 'woody',
 20: 'comically',
 21: 'worked',
 22: 'adorning',
 23: 'boldy',
 24: 'whose',
 25: 'feeding',
 26: 'patches',
 27: 'eligible',
 28: 'ventral',
 29: 'segments',
 30: 'swan',
 31: 'bl',
 32: 'adaptable',
 33: 'supported',
 34: 'undertails',
 35: 'patched',
 36: 'bike',
 37: 'brilliant',
 38: 'buugy',
 39: 'under',
 40: 'regal',
 41: 'downcurved',
 42: 'immature',
 43: 'adabomin',
 44: 'thoart',
 45: 'spec',
 46: 'shrot',
 47: 'alternating',
 48: 'plants',
 49: 'risk',
 50: 'stright',
 51: 'mediium',
 52: 'accentuates',
 53: 'rise',
 54: 'fether',
 55: 'connects',
 56: 'inquisitive',
 57: 'flycatcher',
 58: 'ovals',
 59: 'every',
 60: 'foul',
 61: 'rec',
 62: 'fur',
 63: 'excited',
 64: 'i

# Data generation code

In [ ]:
IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def make_dataset_txt(files):
    """
    :param path_files: the path of txt file that store the image paths
    :return: image paths and sizes
    """
    img_paths = []

    with open(files) as f:
        paths = f.readlines()

    for path in paths:
        path = path.strip()
        img_paths.append(os.path.join(os.path.dirname(files), path))

    return img_paths, len(img_paths)

def make_dataset_dir(dir):
    """
    :param dir: directory paths that store the image
    :return: image paths and sizes
    """
    img_paths = []

    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    for root, _, fnames in os.walk(dir):
        for fname in sorted(fnames):
            if is_image_file(fname):
                path = os.path.join(root, fname)
                img_paths.append(path)

    return img_paths, len(img_paths)

def make_dataset(path_files):
    if os.path.isfile(path_files):
        paths, size = make_dataset_txt(path_files)
    else:
        paths, size = make_dataset_dir(path_files)

    return paths, size


In [ ]:
def random_image(self):
        """Random load the test image"""
        value = self.comboBox.currentIndex()
        if value == 0:
            return
        # read random mask
        if self.opt.mask_file != "none":
            mask_paths, mask_size = make_dataset(self.opt.mask_file)
            item = np.random.randint(0, mask_size - 1)
            self.mname = mask_paths[item]

        item = np.random.randint(0, self.image_size-1)
        self.fname = self.image_paths[item]
        if self.firstly:
            #self.fname = './datasets/CUB_200_2011\\images/042.Vermilion_Flycatcher/Vermilion_Flycatcher_0045_42219.jpg'
            self.fname = './datasets/CUB_200_2011\\images/169.Magnolia_Warbler/Magnolia_Warbler_0063_166121.jpg'

            self.firstly = False
        self.showImage(self.fname)
        print(self.fname)

        img_name = os.path.basename(self.fname)
        caption = sorted(self.captions[img_name], key=lambda x:len(x))[-1]
        self.textEdit.setText(caption)

In [ ]:
def vectorize_captions_idx_batch(batch_padded_captions_idx, batch_length, language_encoder):
    '''Transform batch_padded_captions_idx to sentence embedding'''
    batch_size = len(batch_length)

def tensor2im(image_tensor, bytes=255.0, imtype=np.uint8):
    if image_tensor.dim() == 3:
        image_numpy = image_tensor.cpu().float().numpy()
    else:
        image_numpy = image_tensor[0].cpu().float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * bytes

    return image_numpy.astype(imtype)

def compute_errors(ground_truth, pre):

    # l1 loss
    l1 = np.mean(np.abs(ground_truth-pre))

    # PSNR
    mse = np.mean((ground_truth - pre) ** 2)
    if mse == 0:
        PSNR = 100
    else:
        PSNR = 20 * math.log10(255.0 / math.sqrt(mse))

    # TV
    gx = pre - np.roll(pre, -1, axis=1)
    gy = pre - np.roll(pre, -1, axis=0)
    grad_norm2 = gx ** 2 + gy ** 2
    TV = np.mean(np.sqrt(grad_norm2))

    return l1, PSNR, TV

In [ ]:
import torch.nn.functional as F

def scale_img(img, size):
    scaled_img = F.interpolate(img, size=size, mode='bilinear', align_corners=True)
    return scaled_img

def scale_pyramid(img, num_scales):
    scaled_imgs = [img]

    s = img.size()

    h = s[2]
    w = s[3]

    for i in range(1, num_scales):
        ratio = 2**i
        nh = h // ratio
        nw = w // ratio
        scaled_img = scale_img(img, size=[nh, nw])
        scaled_imgs.append(scaled_img)

    scaled_imgs.reverse()
    return scaled_imgs

In [ ]:
def fill_mask(self):
        """Forward to get the generation results"""
        img_m, img_c, img_truth, mask, text_idx, text_len = self.set_input()
        if self.comboBox.currentIndex() == 0:
            return
        if text_len < 1:
            self.textEdit.setText('Input some words about this bird or the bird you want.')
            return
        print(self.textEdit.toPlainText())
        if self.PaintPanel.iteration < 100:
            print(self.PaintPanel.iteration)
            with torch.no_grad():
                # encoder process
                word_embeddings, sentence_embedding = vectorize_captions_idx_batch(
                                                text_idx, text_len, self.model.text_encoder)
                img_mask = torch.ones_like(img_m)
                img_mask[img_m == 0.] = 0.
                distributions, f, f_text = self.model.net_E(img_m, sentence_embedding, word_embeddings, None, img_mask, img_c)

                variation_factor = 1. if self.checkBox.isChecked() else 0.
                q_distribution = torch.distributions.Normal(distributions[-1][0], distributions[-1][1] * variation_factor)

                z = q_distribution.sample()

                # decoder process
                scale_mask = scale_pyramid(mask, 4)
                self.img_g, _ = self.model.net_G(z, f_text, f_e=f[2], mask=scale_mask[0].chunk(3, dim=1)[0])

                self.img_out = (1 - mask) * self.img_g[-1].detach() + mask * img_m

                # get score
                l1, PSNR, TV = compute_errors(tensor2im(self.img_truth), tensor2im(self.img_out.detach()))

                self.label_6.setText(str(PSNR))

                self.PaintPanel.iteration += 1

        self.show_result_flag = True
        # import ipdb; ipdb.set_trace()
        self.show_result()

In [ ]:
for i in range('''len of dataset'''):
    